#### Setup Project and import necessary libraries

In [1]:
import pandas as pd 
from matplotlib import pyplot as plt 
%matplotlib inline
from sklearn.impute import SimpleImputer
from IPython.display import display 
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt, matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
import gc

In [2]:
# free local memory
x = 5
gc.collect()

0

### Add PLZ to Gas Data

In [3]:
#Import Data (entire data takes about 230 sek)
pathGas = './Data created autom/' + "clean_" + 'competitor_market_prices_gas.csv'

df_compGas = pd.read_csv(pathGas)
#df_compGas = pd.read_csv(pathGas, nrows=1000)

display(df_compGas.head())

,Unnamed: 0,date_valid_to,date_valid_from,post_code,city,energy,consumption_range_kwh,supplier,tariff,rank,price_year_eur
0,0,2021-09-01,2021-05-21,1309,Dresden,gas,1000,Levante,Sawsbuck,1.0,131.30
1,1,2021-09-01,2021-05-21,1309,Dresden,gas,1500,Levante,Sawsbuck,1.0,160.95
2,2,2021-09-01,2021-05-21,1309,Dresden,gas,2000,Levante,Sawsbuck,1.0,190.60
3,3,2021-09-01,2021-05-21,1309,Dresden,gas,2300,Levante,Sawsbuck,1.0,208.39
4,4,2021-09-01,2021-05-21,1309,Dresden,gas,2500,Levante,Sawsbuck,1.0,220.25


#### Import external Data set and explore it

In [4]:
#import external data
df_plzGer = pd.read_csv("./Data own sources/georef-germany-postleitzahl.csv", sep=';')

#split coordinates up in 2 columns
df_plzGer= pd.concat([df_plzGer, df_plzGer['geo_point_2d'].str.split(',', expand=True)], axis=1)
#rename
#df2 = df_plzGer.drop(df_plzGer.iloc[:, 12:13],axis = 1)
#display(df2)
df_plzGer.rename(columns={0: 'Latitude', 1: 'Longitude'},inplace=True)
df_plzGer.head()

display(df_plzGer)

,Name,PLZ Name (short),PLZ Name (long),Geometry,Postleitzahl / Post code,Kreis code,Land name,Land code,Kreis name,geo_point_2d,Latitude,Longitude
0,47551,Bedburg-Hau,47551 Bedburg-Hau,"{""coordinates"": [[[6.1156615, 51.7419192], [6....",47551,5154,Nordrhein-Westfalen,5,Kreis Kleve,"51.7581345416,6.20695861505",51.7581345416,6.20695861505
1,52477,Alsdorf,52477 Alsdorf,"{""coordinates"": [[[6.1218109, 50.8594794], [6....",52477,5334,Nordrhein-Westfalen,5,Kreis Städteregion Aachen,"50.86861665,6.17550818828",50.86861665,6.17550818828
2,52159,Roetgen,52159 Roetgen,"{""coordinates"": [[[6.1661074, 50.6618499], [6....",52159,5334,Nordrhein-Westfalen,5,Kreis Städteregion Aachen,"50.6583845345,6.18337503317",50.6583845345,6.18337503317
3,52156,Monschau,52156 Monschau,"{""coordinates"": [[[6.1740833, 50.5563913], [6....",52156,5334,Nordrhein-Westfalen,5,Kreis Städteregion Aachen,"50.5630704175,6.20863299366",50.5630704175,6.20863299366
4,52152,Simmerath,52152 Simmerath,"{""coordinates"": [[[6.2499715, 50.6363698], [6....",52152,5334,Nordrhein-Westfalen,5,Kreis Städteregion Aachen,"50.6340050941,6.26122056317",50.6340050941,6.26122056317
...,...,...,...,...,...,...,...,...,...,...,...,...
8929,67319,Wattenheim,67319 Wattenheim,"{""coordinates"": [[[7.9483705, 49.4867376], [7....",67319,7332,Rheinland-Pfalz,7,Landkreis Bad Dürkheim,"49.5050957815,8.02168412687",49.5050957815,8.02168412687
8930,67304,Eisenberg (Pfalz),67304 Eisenberg (Pfalz),"{""coordinates"": [[[7.957568, 49.5296522], [7.9...",67304,7333,Rheinland-Pfalz,7,Landkreis Donnersbergkreis,"49.5568522608,8.03801378887",49.5568522608,8.03801378887
8931,65604,Elz,65604 Elz,"{""coordinates"": [[[7.9638058, 50.4120135], [7....",65604,6533,Hessen,6,Landkreis Limburg-Weilburg,"50.4185838408,8.01120492683",50.4185838408,8.01120492683
8932,67466,Lambrecht (Pfalz),67466 Lambrecht (Pfalz),"{""coordinates"": [[[7.9848577, 49.3432291], [7....",67466,7332,Rheinland-Pfalz,7,Landkreis Bad Dürkheim,"49.369532844,8.05504340911",49.369532844,8.05504340911


In [5]:
#explore
df_plzGer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8934 entries, 0 to 8933
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Name                      8934 non-null   int64 
 1   PLZ Name (short)          8934 non-null   object
 2   PLZ Name (long)           8934 non-null   object
 3   Geometry                  8934 non-null   object
 4   Postleitzahl / Post code  8934 non-null   int64 
 5   Kreis code                8934 non-null   int64 
 6   Land name                 8934 non-null   object
 7   Land code                 8934 non-null   int64 
 8   Kreis name                8934 non-null   object
 9   geo_point_2d              8934 non-null   object
 10  Latitude                  8934 non-null   object
 11  Longitude                 8934 non-null   object
dtypes: int64(4), object(8)
memory usage: 837.7+ KB


##### Just select Bundesland and PLZ

In [6]:
df_plzGer_edit = df_plzGer[['Postleitzahl / Post code', 'Land name', 'Latitude', 'Longitude']]
display(df_plzGer_edit)

,Postleitzahl / Post code,Land name,Latitude,Longitude
0,47551,Nordrhein-Westfalen,51.7581345416,6.20695861505
1,52477,Nordrhein-Westfalen,50.86861665,6.17550818828
2,52159,Nordrhein-Westfalen,50.6583845345,6.18337503317
3,52156,Nordrhein-Westfalen,50.5630704175,6.20863299366
4,52152,Nordrhein-Westfalen,50.6340050941,6.26122056317
...,...,...,...,...
8929,67319,Rheinland-Pfalz,49.5050957815,8.02168412687
8930,67304,Rheinland-Pfalz,49.5568522608,8.03801378887
8931,65604,Hessen,50.4185838408,8.01120492683
8932,67466,Rheinland-Pfalz,49.369532844,8.05504340911


#### Merge table (add Bundesland according to PLZ to original data)

In [7]:
#Merge (LEFT Join)
df_merged = pd.merge(df_compGas, df_plzGer_edit, left_on='post_code', right_on='Postleitzahl / Post code', how = 'left')
display(df_merged.head())

,Unnamed: 0,date_valid_to,date_valid_from,post_code,city,energy,consumption_range_kwh,supplier,tariff,rank,price_year_eur,Postleitzahl / Post code,Land name,Latitude,Longitude
0,0,2021-09-01,2021-05-21,1309,Dresden,gas,1000,Levante,Sawsbuck,1.0,131.30,1309,Sachsen,51.0492202614,13.7894318759
1,1,2021-09-01,2021-05-21,1309,Dresden,gas,1500,Levante,Sawsbuck,1.0,160.95,1309,Sachsen,51.0492202614,13.7894318759
2,2,2021-09-01,2021-05-21,1309,Dresden,gas,2000,Levante,Sawsbuck,1.0,190.60,1309,Sachsen,51.0492202614,13.7894318759
3,3,2021-09-01,2021-05-21,1309,Dresden,gas,2300,Levante,Sawsbuck,1.0,208.39,1309,Sachsen,51.0492202614,13.7894318759
4,4,2021-09-01,2021-05-21,1309,Dresden,gas,2500,Levante,Sawsbuck,1.0,220.25,1309,Sachsen,51.0492202614,13.7894318759


#### Validity Checks
#### Do all our PLZ from our data set have a partner in the new data set?

In [8]:
df_merged.isna().any()
## -> result: we do NOT have NaN in our data set -> every PLZ has found a merge partner

Unnamed: 0                  False
date_valid_to               False
date_valid_from             False
post_code                   False
city                        False
energy                      False
consumption_range_kwh       False
supplier                    False
tariff                      False
rank                        False
price_year_eur              False
Postleitzahl / Post code    False
Land name                   False
Latitude                    False
Longitude                   False
dtype: bool

In [9]:
test = df_merged.sort_values(by=['Postleitzahl / Post code'], ascending=False)
test

,Unnamed: 0,date_valid_to,date_valid_from,post_code,city,energy,consumption_range_kwh,supplier,tariff,rank,price_year_eur,Postleitzahl / Post code,Land name,Latitude,Longitude
21809924,18287487,9999-12-31,2022-10-15,99089,Erfurt,gas,500000,Sarma,Ampharos,3.0,146178.08,99089,Thüringen,50.9972843241,11.0148016139
2885372,2418477,2021-09-20,2021-09-18,99089,Erfurt,gas,18000,Papagayos,Skarmory,43.0,1352.25,99089,Thüringen,50.9972843241,11.0148016139
2885374,2418479,2021-09-20,2021-09-18,99089,Erfurt,gas,18000,Sarma,Aegislash,19.0,1181.80,99089,Thüringen,50.9972843241,11.0148016139
2885375,2418480,2021-09-20,2021-09-18,99089,Erfurt,gas,18000,Sarma,Ampharos,25.0,1189.80,99089,Thüringen,50.9972843241,11.0148016139
2885376,2418481,2021-09-20,2021-09-18,99089,Erfurt,gas,18000,Septentrio,Thundurus,26.0,1192.31,99089,Thüringen,50.9972843241,11.0148016139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3026916,2535674,2021-09-21,2021-09-21,1309,Dresden,gas,3000,Aurassos,Yamask,50.0,416.28,1309,Sachsen,51.0492202614,13.7894318759
3026915,2535673,2021-09-21,2021-09-21,1309,Dresden,gas,3000,Aspr,Maractus,27.0,351.60,1309,Sachsen,51.0492202614,13.7894318759
3026914,2535672,2021-09-21,2021-09-21,1309,Dresden,gas,2500,Willy,Shuppet,39.0,343.52,1309,Sachsen,51.0492202614,13.7894318759
3026913,2535671,2021-09-21,2021-09-21,1309,Dresden,gas,2500,Tornado,Staraptor,48.5,370.85,1309,Sachsen,51.0492202614,13.7894318759


In [10]:
test = df_merged.sort_values(by=['Postleitzahl / Post code'], ascending=True)
test

test=5
gc.collect()

20

In [11]:
# do we have same amount of values for PLZ in our data set and the external one?
print(df_merged.nunique())
print("")
print(df_plzGer.nunique())

## -> result: We only have the 100 top postcodes included in data set (as described in our task by EON)

Unnamed: 0                  18287488
date_valid_to                    302
date_valid_from                  355
post_code                        100
city                             102
energy                             1
consumption_range_kwh             32
supplier                         125
tariff                           240
rank                            1163
price_year_eur               1740182
Postleitzahl / Post code         100
Land name                         16
Latitude                         116
Longitude                        116
dtype: int64

Name                        8168
PLZ Name (short)            6415
PLZ Name (long)             8168
Geometry                    8706
Postleitzahl / Post code    8168
Kreis code                   401
Land name                     16
Land code                     16
Kreis name                   401
geo_point_2d                8706
Latitude                    8706
Longitude                   8706
dtype: int64


In [12]:
df_merged = df_merged.drop('Postleitzahl / Post code', axis=1)
#rename
df_merged.rename(columns={'Land name': 'Bundesland'}, inplace=True)
df_merged.head()

,Unnamed: 0,date_valid_to,date_valid_from,post_code,city,energy,consumption_range_kwh,supplier,tariff,rank,price_year_eur,Bundesland,Latitude,Longitude
0,0,2021-09-01,2021-05-21,1309,Dresden,gas,1000,Levante,Sawsbuck,1.0,131.30,Sachsen,51.0492202614,13.7894318759
1,1,2021-09-01,2021-05-21,1309,Dresden,gas,1500,Levante,Sawsbuck,1.0,160.95,Sachsen,51.0492202614,13.7894318759
2,2,2021-09-01,2021-05-21,1309,Dresden,gas,2000,Levante,Sawsbuck,1.0,190.60,Sachsen,51.0492202614,13.7894318759
3,3,2021-09-01,2021-05-21,1309,Dresden,gas,2300,Levante,Sawsbuck,1.0,208.39,Sachsen,51.0492202614,13.7894318759
4,4,2021-09-01,2021-05-21,1309,Dresden,gas,2500,Levante,Sawsbuck,1.0,220.25,Sachsen,51.0492202614,13.7894318759


In [14]:
pathGasWithPLZ = './Data created autom/' + "clean_PLZ_" + 'competitor_market_prices_gas.csv'
df_merged.to_csv(pathGasWithPLZ, sep=',', index=False)


In [15]:
df_merged = 5
gc.collect()

0

In [16]:
pathPower= './Data created autom/' + "clean_" + 'competitor_market_prices_power.csv'

#Import data
df_compPow= pd.read_csv(pathPower)
#df_compPow= pd.read_csv(pathPower, nrows=100000)

#Merge
df_merged2 = pd.merge(df_compPow, df_plzGer_edit, left_on='post_code', right_on='Postleitzahl / Post code', how = 'left')
#display(df_merged2.head())

In [17]:
df_merged2.isna().any()

Unnamed: 0                  False
date_valid_to               False
date_valid_from             False
post_code                   False
city                        False
energy                      False
consumption_range_kwh       False
supplier                    False
tariff                      False
rank                        False
price_year_eur              False
Postleitzahl / Post code    False
Land name                   False
Latitude                    False
Longitude                   False
dtype: bool

In [18]:
# do we have same amount of values for PLZ in our data set and the external one?
print(df_merged2.nunique())
print("")
print(df_plzGer.nunique())

## -> result: We only have the 100 top postcodes included in data set (as described in our task by EON)

Unnamed: 0                  33161806
date_valid_to                    308
date_valid_from                  357
post_code                        100
city                             102
energy                             1
consumption_range_kwh             30
supplier                         140
tariff                           296
rank                            1875
price_year_eur               1516403
Postleitzahl / Post code         100
Land name                         16
Latitude                         116
Longitude                        116
dtype: int64

Name                        8168
PLZ Name (short)            6415
PLZ Name (long)             8168
Geometry                    8706
Postleitzahl / Post code    8168
Kreis code                   401
Land name                     16
Land code                     16
Kreis name                   401
geo_point_2d                8706
Latitude                    8706
Longitude                   8706
dtype: int64


In [19]:
df_merged2 = df_merged2.drop('Postleitzahl / Post code', axis=1)
#rename
df_merged2.rename(columns={'Land name': 'Bundesland'}, inplace=True)
df_merged2.head()

,Unnamed: 0,date_valid_to,date_valid_from,post_code,city,energy,consumption_range_kwh,supplier,tariff,rank,price_year_eur,Bundesland,Latitude,Longitude
0,0,2021-09-01,2021-06-02,14480,Potsdam,power,3500,Galerne,Skiploom,1.0,878.86,Brandenburg,52.3742825809,13.1346615232
1,1,2021-09-01,2021-06-09,14480,Potsdam,power,2800,Galerne,Skiploom,1.0,678.69,Brandenburg,52.3742825809,13.1346615232
2,2,2021-09-01,2021-07-10,14480,Potsdam,power,7500,Galerne,Skiploom,1.0,2022.68,Brandenburg,52.3742825809,13.1346615232
3,3,2021-09-01,2021-07-10,14480,Potsdam,power,7800,Galerne,Skiploom,1.0,2108.47,Brandenburg,52.3742825809,13.1346615232
4,4,2021-09-01,2021-07-10,14480,Potsdam,power,8000,Galerne,Skiploom,1.0,2165.66,Brandenburg,52.3742825809,13.1346615232


In [20]:
pathPowerWithPLZ= './Data created autom/' + "clean_PLZ_" + 'competitor_market_prices_power.csv'

df_merged2.to_csv(pathPowerWithPLZ, sep=',', index=False)

In [21]:
check = pd.read_csv('./Data created autom/df_merged_Gas.csv')
display(check)

,date_valid_from,date_valid_to,rank,consumption_range_kwh,tariff,supplier,price_year_eur,post_code,city,energy,Bundesland
0,2021-09-01,2021-09-01,1,22500,Shuppet,Willy,1340.77,58636,Iserlohn,gas,Nordrhein-Westfalen
1,2021-08-31,2021-09-01,33,12000,Shuckle,Jugo,906.78,58636,Iserlohn,gas,Nordrhein-Westfalen
2,2021-08-31,2021-09-01,15,40000,Hoopa,Kachchab,2599.79,58636,Iserlohn,gas,Nordrhein-Westfalen
3,2021-08-31,2021-09-01,19,22500,Pidgeot,Criador,1515.00,58636,Iserlohn,gas,Nordrhein-Westfalen
4,2021-08-31,2021-09-01,20,15000,Wigglytuff,Leste,975.59,58636,Iserlohn,gas,Nordrhein-Westfalen
...,...,...,...,...,...,...,...,...,...,...,...
1223,2021-08-31,2021-09-01,18,20000,Natu,Criador,1404.00,18109,Rostock,gas,Mecklenburg-Vorpommern
1224,2021-08-31,2021-09-01,25,200000,Giratina,Gegenwind,13497.00,18109,Rostock,gas,Mecklenburg-Vorpommern
1225,2021-08-28,2021-09-01,24,17000,Pawmi,Oder,1153.29,18109,Rostock,gas,Mecklenburg-Vorpommern
1226,2021-08-28,2021-09-01,17,3000,Pidgeot,Criador,309.10,18109,Rostock,gas,Mecklenburg-Vorpommern
